Install relevant modules and packages

In [1]:
!apt-get install -y dcm2niix # '!' runs commands in the console
!apt-get install -y parallel
!apt-get install -y wget
!apt-get install -y grep
!apt-get install -y tcsh

!pip install nibabel

import os
import glob
import nibabel as nib

# install freesurfer (might take a few minutes)

# if you are having issues downloading freesurfer, it is likely due to the version selected here
!wget -O freesurfer.tar.gz https://freesurfer.net/pub/dist/freesurfer/7.5.0/freesurfer-macOS-darwin_arm64-7.5.0.tar.gz
!tar -xzf freesurfer.tar.gz

# set the relevant freesurfer directories
os.environ['FREESURFER_HOME'] = '/content/freesurfer'
os.environ['SUBJECTS_DIR'] = '/content/subjects'
os.environ['FS_LICENSE'] = '/content/license.txt'  # Add your license file path if you have one
os.environ['PATH'] += ':/content/freesurfer/bin'

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libyaml-cpp0.7 pigz
The following NEW packages will be installed:
  dcm2niix libyaml-cpp0.7 pigz
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 417 kB of archives.
After this operation, 1,393 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 pigz amd64 2.6-1 [63.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libyaml-cpp0.7 amd64 0.7.0+dfsg-8build1 [97.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 dcm2niix amd64 1.0.20211006-1build1 [256 kB]
Fetched 417 kB in 0s (3,100 kB/s)
Selecting previously unselected package pigz.
(Reading database ... 123622 files and directories currently installed.)
Preparing to unpack .../archives/pigz_2.6-1_amd64.deb ...
Unpacking pigz (2.6-1) ...
Selecting previously unselected package libyaml-c

Get anonymized (T1 weighted) dicom files for analysis, and create temporary folders to store these and other files.

In [2]:
# clone the repository to get the raw dicoms
!git clone https://github.com/datalad/example-dicom-structural

# create folders for the NIfTI conversions and corresponding recons
!mkdir /content/nii_files/
!mkdir /content/freesurfer_output/

'''
Note: "T1 weighting" refers to a form of MRI image processing performed by technicians.
Online resources will nearly always provide processed MRI images,
though they may not be T1 weighted
''';

Cloning into 'example-dicom-structural'...
remote: Enumerating objects: 393, done.
remote: Total 393 (delta 0), reused 0 (delta 0), pack-reused 393 (from 1)
Receiving objects: 100% (393/393), 15.45 MiB | 17.66 MiB/s, done.
Resolving deltas: 100% (223/223), done.


Convert the DICOMs to NIfTIs

In [7]:
# prepare paths
INPUT_PATH = '/content/example-dicom-structural/dicoms/'
OUTPUT_PATH = '/content/nii_files/'

# grep to ignore some warnings regarding the manufacturer (since we're using sample dicoms)
!dcm2niix -o '{OUTPUT_PATH}' '{INPUT_PATH}' | grep -v "Unknown manufacturer"

Chris Rorden's dcm2niiX version v1.0.20211006  (JP2:OpenJPEG) GCC11.2.0 x86-64 (64-bit Linux)
Found 384 DICOM file(s)
Convert 384 DICOM as /content/nii_files/dicoms_anat-T1w_20130717141500_401a (274x384x384x1)
Conversion required 0.532300 seconds (0.531181 for core code).


Reconstruct the NIfTI files.

Parallel allows you to recon multiple subjects simultaneously by recruiting n CPU cores (determined by --jobs n)

In [9]:
# prepare paths
INPUT_PATH = '/content/drive/MyDrive/freesurfer_output'
nii_paths = glob.glob('/content/drive/MyDrive/nii_files/*')
ALL_NIIS = ' '.join(nii_paths) # format that parallel wants

# Use parallel to execute recon-all on each NIfTI file
!parallel --jobs 2 'recon-all -i {} -sd {} -all' ::: $ALL_NIIS ::: "$INPUT_PATH"

/bin/bash: /content/freesurfer/bin/recon-all: /bin/tcsh: bad interpreter: No such file or directory
